# <center>基于问答数据的特征工程

- **Step 1. 确定推荐系统的需求**

&emsp;&emsp;开发一个推荐系统的功能链路，通过分析用户的历史对话记录，智能推荐最适合用户需求的课程。

- **Step 2. 数据说明**

&emsp;&emsp;我们公司的课程分为四大类别，每类课程都通过PDF课件提供详细的教学内容，其中涵盖的技术领域：

- 机器学习：涵盖从基础理论到高级应用的各个方面。
- 深度学习：介绍深度学习的基础架构、算法及其在各行业中的应用。
- 大数据架构：通过大数据平台架构设计从而解决复杂大数据场景。
- 大模型工程落地学习计划：基于大模型进行应用开发。

&emsp;&emsp;每个PDF课件包含多个一级和二级标题，这些标题明确区分不同的知识点，便于学习和复习：

- 一级标题：代表主要的知识模块或课程大纲的核心部分。
- 二级标题：在每个主要知识模块下细分更具体的话题或概念。

- **Step 3. 数据处理**

&emsp;&emsp;当大模型介入到数据处理流程的时候，我们需要考虑的就如RAG的Indexing过程一样：如何找到一种最合适的文本读取方式，更好的区分各个模块、内容，以保证收集到的数据都是相对完整且独立的。

&emsp;&emsp;这里我们推荐使用的是：Markdown 格式。 Markdown是一种轻量级标记语言，允许用户使用简单的语法格式化纯文本。它广泛用于创建结构化文档，特别是在 GitHub、Jupyter Notebook 和各种内容管理系统等平台上。当将数据输入 LLM 或 RAG 系统时，使用 Markdown 格式有几个好处：

1. 结构化内容：Markdown 会将内容组织为标题、列表、表格和其他结构化元素。这种结构有助于更好地理解和上下文保存。
2. 富文本：Markdown 支持基本格式，例如粗体、斜体、链接和代码块。在输入数据中包含富文本可以增强语言模型的上下文。
3. Markdown 允许嵌入超链接、脚注和参考。在构建元数据过程中，对于引用外部资源或提供额外的上下文至关重要。

&emsp;&emsp;这里，我们给大家推荐一个PDF --> Markdown的转换工具：PyMuPDF。官方：https://github.com/pymupdf/RAG

&emsp;&emsp;PyMuPDF能够从 PDF 页面中提取文本、图像、矢量图形，以及自 2023 年 8 月起还能够提取表格。每种对象类型都有自己的提取方法：一种用于文本，另一种用于表格、图像和矢量图形，同时合并了这些不同的提取，以生成一个通用的、统一的 Markdown 字符串，该字符串一致地表示整个页面的内容。

In [1]:
 ! pip install -U pymupdf4llm

  Using cached PyMuPDFb-1.24.10-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 118.6 kB/s eta 0:00:0000:0100:01
Using cached PyMuPDFb-1.24.10-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (15.9 MB)


In [2]:
import pymupdf4llm

doc = pymupdf4llm.to_markdown("data/Ch 1 开源大模型本地部署硬件指南.pdf")

Processing data/Ch 1 开源大模型本地部署硬件指南.pdf...
[                                        ] (0/2[=                                       ] ( 1/2=[===                                     ] ( 2/2=[=====                                   ] ( 3/2=[=======                                 ] ( 4/2=[=========                               ] ( 5/2[==========                              ] ( 6/22=[============                            ] ( 7/22=[==============                          ] ( 8/22=[================                        ] ( 9/22=[==================                      ] (10/22[====================                    ] (11/22=[=====================                   ] (12/2=[=======================                 ] (13/2=[=========================               ] (14/2=[===========================             ] (15/2=[=============================           ] (16/2[==============================          ] (17/22=[================================        ] (18/22=[=======================

In [3]:
doc

'# 本地部署开源大模型\n\n## Ch.1 如何选择合适的硬件配置\n\n为了在本地有效部署和使用开源大模型，深入理解硬件与软件的需求至关重要。在硬件需求方面，关键\n\n是配置一台或多台高性能的个人计算机系统或租用配备了先进GPU的在线服务器，确保有足够的内存和存储\n\n空间来处理大数据和复杂模型。至于软件需求，推荐使用Ubuntu操作系统，因其在机器学习领域的支持和\n\n兼容性优于Windows。编程语言建议以Python为主，结合TensorFlow或PyTorch等流行机器学习框架，并\n\n利用DeepSpeed等优化工具来提升大模型的运行效率和性能。\n\n所以在本系列课程中，我们将从硬件选择入手，逐步引导大家理解并掌握如何为大模型部署选择合适的\n\n硬件，以及如何高效地配置和运行这些模型，从零到一实现大模型的本地部署和应用。首先来看硬件方面，\n\n提前规划计算资源是必要的。目前，我们主要考虑以下两种途径：\n\n1. 配置个人计算机或服务器，组建一个适合大模型使用需求的计算机系统。\n\n2. 租用在线GPU服务，通过云计算平台获取大模型所需的计算能力。\n\n# 一、大模型应用需求分析\n\n大模型的本地部署主要应用于三个方面：训练（train）、高效微调（fine-tune）和推理\n\n**（inference）。这些过程在算力消耗上有显著差异：**\n\n**训练：算力最密集，通常消耗的算力是推理过程的至少三个数量级以上。**\n\n**微调：微调是在预训练模型的基础上对其进行进一步调整以适应特定任务的过程，其算力需求低于训**\n\n练，但高于推理。\n\n**推理：推理指的是使用训练好的模型来进行预测或分析，是算力消耗最低的阶段。**\n\n总的来说，在算力消耗上，训练 > 微调 > 推理。\n\n从头训练一个大模型并非易事，这不仅对个人用户，对于许多企业而言也同样困难。因此，如果个人使\n\n用，关注点应该放在推理和微调的性能上。在这两种应用需求下，对硬件的核心要求体现在GPU的选择上，\n\n**对CPU和内存的要求并不高。无论是选择租用在线算力还是配置本地计算机，如果想在本地运行大模型，我**\n\n们可以拆分成两个关注点：\n\n模型：选择什么基座模型或微调模型，这可以直接下载至本地。\n\n硬件：希望在什么硬件平台上来

&emsp;&emsp;接下来，我们定义一个用于从Markdown格式文本中提取目录（Table of Contents，TOC）的Python函数。它使用正则表达式来识别Markdown中的标题并将其分类。代码如下：

In [4]:
import re

def extract_toc(markdown):
    # 初始化一个列表用于存储目录项，每项为标题级别和标题文本的元组
    toc = []
    
    # 正则表达式模式，用于匹配Markdown标题，这包括'#'字符和后面的文本
    # '^' 表示行的开始
    # '(#+)' 捕获一个或多个'#'字符
    # '\s+' 匹配一个或多个空格
    # '(.*)' 捕获标题文本，直到行尾
    pattern = r'^(#+)\s+(.*)$'
    
    # 将输入的Markdown文本按行分割
    lines = markdown.split('\n')
    
    # 遍历每一行
    for line in lines:
        # 使用正则表达式匹配当前行
        match = re.match(pattern, line)
        
        # 如果当前行匹配Markdown标题格式
        if match:
            # 计算标题的级别，基于'#'的数量减去1（即，'#'对应0级，'##'对应1级，以此类推）
            level = len(match.group(1)) - 1
            
            # 获取标题文本
            title = match.group(2)
            
            # 将当前标题的级别和文本作为元组添加到目录列表中
            toc.append((level, title))
    
    # 返回整理好的目录列表
    return toc

&emsp;&emsp;执行调用测试函数功能：

In [6]:
toc=extract_toc(doc)
toc

[(0, '本地部署开源大模型'),
 (1, 'Ch.1 如何选择合适的硬件配置'),
 (0, '一、大模型应用需求分析'),
 (0, '二、硬件配置的选择标准'),
 (1, '2.1 选择满足显存需求的 GPU'),
 (1, '2.2 主流显卡性能分析'),
 (1, '2.3 单卡4090 vs A100系列'),
 (1, '2.4 单卡4090 vs 双卡3090'),
 (1, '2.5 风扇卡与涡轮卡如何选择'),
 (1, '2.6 整机参考配置'),
 (1, '2.7 显卡博弈的形式分析'),
 (1, '2.8 国产AI超算芯片期待'),
 (0, '三、组装计算机硬件选型策略'),
 (1, '3.1 GPU选型策略'),
 (1, '3.2 CPU选型策略'),
 (1, '3.3 散热选型策略'),
 (1, '3.5 硬盘选型策略'),
 (1, '3.7 电源选型策略'),
 (1, '3.8 机箱选型策略')]

In [7]:
toc = extract_toc(doc)

for level, title in toc:
    print(' ' * level + title)

本地部署开源大模型
 Ch.1 如何选择合适的硬件配置
一、大模型应用需求分析
二、硬件配置的选择标准
 2.1 选择满足显存需求的 GPU
 2.2 主流显卡性能分析
 2.3 单卡4090 vs A100系列
 2.4 单卡4090 vs 双卡3090
 2.5 风扇卡与涡轮卡如何选择
 2.6 整机参考配置
 2.7 显卡博弈的形式分析
 2.8 国产AI超算芯片期待
三、组装计算机硬件选型策略
 3.1 GPU选型策略
 3.2 CPU选型策略
 3.3 散热选型策略
 3.5 硬盘选型策略
 3.7 电源选型策略
 3.8 机箱选型策略


&emsp;&emsp;进一步，从Markdown格式的文本中根据一级标题提取并组织各个部分的内容。代码如下：

In [8]:
def extract_content_by_sections(markdown):
    # 初始化一个字典来存储每个一级标题及其对应的内容
    sections = {}
    # 用于追踪当前处理的一级标题
    current_section = None
    # 内容累积器，用于收集当前一级标题下的所有行
    content_accumulator = []
    
    # 将输入的Markdown文本按行分割
    lines = markdown.split('\n')
    # 正则表达式模式，用于匹配Markdown标题
    pattern = r'^(#+)\s+(.*)$'
    
    # 遍历每一行
    for line in lines:
        # 使用正则表达式匹配当前行
        match = re.match(pattern, line)
        if match:
            # 计算标题的级别，基于'#'的数量减去1
            level = len(match.group(1)) - 1
            # 获取标题文本
            title = match.group(2)
            # 如果是一级标题
            if level == 0:
                # 如果不是处理的第一个一级标题，存储之前标题下的所有累积内容
                if current_section is not None:
                    sections[current_section] = "\n".join(content_accumulator).strip()
                # 更新当前处理的一级标题并重置内容累积器
                current_section = title
                content_accumulator = []
            elif current_section is not None:
                # 如果是子标题或内容，并且已经有一级标题被处理，继续累积内容
                content_accumulator.append(line)
        elif current_section is not None:
            # 如果当前行不是标题，但已经存在一级标题，继续累积内容
            content_accumulator.append(line)
    
    # 处理完成所有行后，确保最后一个一级标题下的内容也被存储
    if current_section is not None:
        sections[current_section] = "\n".join(content_accumulator).strip()
    
    # 返回存储了各个一级标题和对应内容的字典
    return sections


&emsp;&emsp;这个函数通过Markdown中的标题级别来组织内容，仅收集一级标题下的内容。它忽略不属于任何一级标题下的内容，并将每个一级标题下的内容组织为连续的文本块。

In [9]:
# 调用函数并提取内容
sections = extract_content_by_sections(doc)

# 打印每个一级标题下的内容
for section, content in sections.items():
    print(f"Section: {section}")
    print(content)
    print("\n---\n")

Section: 本地部署开源大模型
## Ch.1 如何选择合适的硬件配置

为了在本地有效部署和使用开源大模型，深入理解硬件与软件的需求至关重要。在硬件需求方面，关键

是配置一台或多台高性能的个人计算机系统或租用配备了先进GPU的在线服务器，确保有足够的内存和存储

空间来处理大数据和复杂模型。至于软件需求，推荐使用Ubuntu操作系统，因其在机器学习领域的支持和

兼容性优于Windows。编程语言建议以Python为主，结合TensorFlow或PyTorch等流行机器学习框架，并

利用DeepSpeed等优化工具来提升大模型的运行效率和性能。

所以在本系列课程中，我们将从硬件选择入手，逐步引导大家理解并掌握如何为大模型部署选择合适的

硬件，以及如何高效地配置和运行这些模型，从零到一实现大模型的本地部署和应用。首先来看硬件方面，

提前规划计算资源是必要的。目前，我们主要考虑以下两种途径：

1. 配置个人计算机或服务器，组建一个适合大模型使用需求的计算机系统。

2. 租用在线GPU服务，通过云计算平台获取大模型所需的计算能力。

---

Section: 一、大模型应用需求分析
大模型的本地部署主要应用于三个方面：训练（train）、高效微调（fine-tune）和推理

**（inference）。这些过程在算力消耗上有显著差异：**

**训练：算力最密集，通常消耗的算力是推理过程的至少三个数量级以上。**

**微调：微调是在预训练模型的基础上对其进行进一步调整以适应特定任务的过程，其算力需求低于训**

练，但高于推理。

**推理：推理指的是使用训练好的模型来进行预测或分析，是算力消耗最低的阶段。**

总的来说，在算力消耗上，训练 > 微调 > 推理。

从头训练一个大模型并非易事，这不仅对个人用户，对于许多企业而言也同样困难。因此，如果个人使

用，关注点应该放在推理和微调的性能上。在这两种应用需求下，对硬件的核心要求体现在GPU的选择上，

**对CPU和内存的要求并不高。无论是选择租用在线算力还是配置本地计算机，如果想在本地运行大模型，我**

们可以拆分成两个关注点：

模型：选择什么基座模型或微调模型，这可以直接下载至本地。

硬件：希望在什么硬件平台上来执行，可以分为 CPU 和 GPU 两大类。

⼤部分开源⼤模型⽀持

&emsp;&emsp;接下来构建推荐系统的项目表示，针对我们的当前的设计，可以考虑将每个大课程分解为多个模块或课件，每个模块代表课程中的一个知识点或一组知识点。这样，每个模块都可以作为推荐系统中的一个独立单元。

&emsp;&emsp;数据字段设计
- ModuleID: 每个模块的唯一标识符。
- Course: 对应的大课程名，如“机器学习”、“自然语言处理”等。
- ModuleName: 模块名称，如“监督学习基础”、“卷积神经网络介绍”等。
- Description: 模块的描述，简要概述模块内容。
- URL: 指向模块资源的链接，如视频讲解、课件下载等。
- Tags: 模块涉及的关键词或技术点，如“线性回归”，“BERT”，“图像分类”等。

In [12]:
import os
def list_pdf_files(directory):
    # 列出指定目录下的所有文件和文件夹
    items = os.listdir(directory)
    # 过滤出以 .pdf 结尾的文件
    pdf_files = [item for item in items if item.endswith('.pdf')]
    return pdf_files

# 使用当前目录作为参数调用函数
current_directory = 'data'
pdf_files = list_pdf_files(current_directory)
print(pdf_files)

['Ch 1 开源大模型本地部署硬件指南.pdf']


In [15]:
import os

def list_pdf_files_full_path(directory):
    # 列出指定目录下的所有文件和文件夹
    items = os.listdir(directory)
    # 使用列表推导式，过滤出以 .pdf 结尾的文件，并为每个文件构造完整路径
    pdf_files = [os.path.join(directory, item) for item in items if item.endswith('.pdf')]
    return pdf_files

# 使用当前目录作为参数调用函数
current_directory = 'data'
pdf_files_full_paths = list_pdf_files_full_path(current_directory)
print(pdf_files_full_paths)

['data/Ch 1 开源大模型本地部署硬件指南.pdf']


# 测试

## 测试

### 测试

In [16]:
import uuid
import os

data = []

for module_name, content in sections.items():
    tags = [line.strip() for line in content.split('\n') if line.startswith('##')]
    tags = [re.sub(r'^##\s+', '', tag) for tag in tags]  # 清理 '##'

    data.append({
        'ModuleID': str(uuid.uuid4()),
        'Course':pdf_files,
        'URL': pdf_files_full_paths,
        'ModuleName': module_name,
        'Tags': ", ".join(tags),
        'Content': content
    })

In [17]:
data

[{'ModuleID': '3fb9c842-26df-4c66-a865-030ffbc05ba3',
  'Course': ['Ch 1 开源大模型本地部署硬件指南.pdf'],
  'URL': ['data/Ch 1 开源大模型本地部署硬件指南.pdf'],
  'ModuleName': '本地部署开源大模型',
  'Tags': 'Ch.1 如何选择合适的硬件配置',
  'Content': '## Ch.1 如何选择合适的硬件配置\n\n为了在本地有效部署和使用开源大模型，深入理解硬件与软件的需求至关重要。在硬件需求方面，关键\n\n是配置一台或多台高性能的个人计算机系统或租用配备了先进GPU的在线服务器，确保有足够的内存和存储\n\n空间来处理大数据和复杂模型。至于软件需求，推荐使用Ubuntu操作系统，因其在机器学习领域的支持和\n\n兼容性优于Windows。编程语言建议以Python为主，结合TensorFlow或PyTorch等流行机器学习框架，并\n\n利用DeepSpeed等优化工具来提升大模型的运行效率和性能。\n\n所以在本系列课程中，我们将从硬件选择入手，逐步引导大家理解并掌握如何为大模型部署选择合适的\n\n硬件，以及如何高效地配置和运行这些模型，从零到一实现大模型的本地部署和应用。首先来看硬件方面，\n\n提前规划计算资源是必要的。目前，我们主要考虑以下两种途径：\n\n1. 配置个人计算机或服务器，组建一个适合大模型使用需求的计算机系统。\n\n2. 租用在线GPU服务，通过云计算平台获取大模型所需的计算能力。'},
 {'ModuleID': 'a4d419bf-68cd-47a7-9efb-bbdef5b8756e',
  'Course': ['Ch 1 开源大模型本地部署硬件指南.pdf'],
  'URL': ['data/Ch 1 开源大模型本地部署硬件指南.pdf'],
  'ModuleName': '一、大模型应用需求分析',
  'Tags': '',
  'Content': '大模型的本地部署主要应用于三个方面：训练（train）、高效微调（fine-tune）和推理\n\n**（inference）。这些过程在算力消耗上有显著差异：**\n\n**训练：算力最密集，通常消耗的算力是推理过程的至

&emsp;&emsp;保存为本地的.csv文件。

In [18]:
import os

# 打印当前工作目录
print("当前工作目录:", os.getcwd())

当前工作目录: /root/project/yanshi


In [20]:
! pip install pandas

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.6 kB/s eta 0:00:0000:0100:05
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [21]:
import pandas as pd

df = pd.DataFrame(data)
csv_file_path = os.path.join('.', 'output.csv')

# 保存 DataFrame 到 CSV 文件
df.to_csv(csv_file_path, index=False)

&emsp;&emsp;接下来我们要做两件事：
1. 依次遍历这个data数据的 Content 字段，调用一个大模型，对这段文本生成摘要，然后填充一个新的字段：Abstract
2. 同时判断 Tags 是否为空，如何为空的话，提取 URL 相同的 Tags中的内容作为提示，让大模型生成Tags

In [23]:
! pip install zhipuai

  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached PyJWT-2.8.0-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 626.5 kB/s eta 0:00:00a 0:00:01
Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
Using cached PyJWT-2.8.0-py3-none-any.whl (22 kB)


In [24]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="2c397e8089af28761792c52d334e92c3.fZQHXOprjyAR3Xs7") # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {"role": "user", "content": "作为一名营销专家，请为智谱开放平台创作一个吸引人的slogan"},
    ],
)
print(response.choices[0].message.content)   

"智谱未来，开放共赢 —— 携手创造无限可能！"


In [25]:
import pandas as pd
from zhipuai import ZhipuAI


client = ZhipuAI(api_key="2c397e8089af28761792c52d334e92c3.fZQHXOprjyAR3Xs7")

# 假设用于生成摘要的函数
def generate_summary(text):
    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=[{"role": "user", "content": "作为一名语言学专家，请根据如下的输入文本:\n{} \n生成一段摘要".format(text)}]
    )
    
    return response.choices[0].message.content

# 假设这是你用于生成 Tags 的函数
def generate_tags(prompt):
    
    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=[{"role": "user", "content": "作为一名语言学专家，请根据类似的示例:\n{} \n生成这段文本的关键性标签，注意：一定要简短且能抓住重点".format(prompt)}]
    )
    
    return response.choices[0].message.content

In [26]:
# 加载 CSV 文件
df = pd.read_csv("output.csv")

# 遍历 DataFrame，生成摘要
df['Abstract'] = df['Content'].apply(generate_summary)

# 检查并填充空的 Tags
for index, row in df.iterrows():
    if pd.isna(row['Tags']) or row['Tags'].strip() == "":
        # 查找具有相同 URL 的其他记录的 Tags
        similar_tags = df[df['URL'] == row['URL']]['Tags'].dropna()
        if not similar_tags.empty:
            tag_prompt = similar_tags.iloc[0]  # 使用第一个非空标签作为提示
            generated_tags = generate_tags(tag_prompt)
            df.at[index, 'Tags'] = generated_tags

In [27]:
df

,ModuleID,Course,URL,ModuleName,Tags,Content,Abstract
0,3fb9c842-26df-4c66-a865-030ffbc05ba3,['Ch 1 开源大模型本地部署硬件指南.pdf'],['data/Ch 1 开源大模型本地部署硬件指南.pdf'],本地部署开源大模型,Ch.1 如何选择合适的硬件配置,## Ch.1 如何选择合适的硬件配置\n\n为了在本地有效部署和使用开源大模型，深入理解硬...,本段文本主要介绍了为了有效部署和使用开源大模型，理解和掌握硬件与软件需求的重要性。建议使用高...
1,a4d419bf-68cd-47a7-9efb-bbdef5b8756e,['Ch 1 开源大模型本地部署硬件指南.pdf'],['data/Ch 1 开源大模型本地部署硬件指南.pdf'],一、大模型应用需求分析,"""硬件选择指南"" 或 ""配置优化要点""",大模型的本地部署主要应用于三个方面：训练（train）、高效微调（fine-tune）和推理...,本地部署大型模型主要涉及训练、高效微调和推理三个环节，其中算力消耗依次递减。由于从头训练大模...
2,ad5870ff-83e3-4858-b9c5-613e7f2ad695,['Ch 1 开源大模型本地部署硬件指南.pdf'],['data/Ch 1 开源大模型本地部署硬件指南.pdf'],二、硬件配置的选择标准,"2.1 选择满足显存需求的 GPU, 2.2 主流显卡性能分析, 2.3 单卡4090 vs...",无论是个人使用、科研团队进行项目研究，还是企业寻求商业应用的落地，不同的应用场景和目标任务\...,本文介绍了选择GPU时需要考虑的因素，包括显存容量、算力、内存带宽等。NVIDIA的GPU在...
3,96fe82ff-62b5-4a2d-bbe3-817854f96b22,['Ch 1 开源大模型本地部署硬件指南.pdf'],['data/Ch 1 开源大模型本地部署硬件指南.pdf'],三、组装计算机硬件选型策略,"3.1 GPU选型策略, 3.2 CPU选型策略, 3.3 散热选型策略, 3.5 硬盘选型...",计算机八大件：CPU、散热器、主板、内存、硬盘、显卡、电源、风扇、机箱，我们依次来看对于一套...,本文详细介绍了如何配置一台用于部署大模型的个人计算机，包括GPU、CPU、散热器、主板、内存...


In [28]:
# 保存更新后的 DataFrame 到新的 CSV 文件
df.to_csv("updated_output.csv", index=False)

In [29]:
df_course = pd.read_csv(
    'updated_output.csv',
    sep=',',
    header=0)     

&emsp;&emsp;然后，新增embedding_info列，用于构建Embedding向量表示。

In [30]:
# 新增 embedding_info_info 列，合并`类别、子类别、标题、摘要`字符串
concatenation_order = ["Course", "ModuleName", "Abstract"]

embedding_info_list = df_course['embedding_info'] = df_course.apply(lambda row: '{ ' + ', '.join(
    f'"{col}": "{row[col]}"' for col in concatenation_order) + ' }', axis=1)

df_course

,ModuleID,Course,URL,ModuleName,Tags,Content,Abstract,embedding_info
0,3fb9c842-26df-4c66-a865-030ffbc05ba3,['Ch 1 开源大模型本地部署硬件指南.pdf'],['data/Ch 1 开源大模型本地部署硬件指南.pdf'],本地部署开源大模型,Ch.1 如何选择合适的硬件配置,## Ch.1 如何选择合适的硬件配置\n\n为了在本地有效部署和使用开源大模型，深入理解硬...,本段文本主要介绍了为了有效部署和使用开源大模型，理解和掌握硬件与软件需求的重要性。建议使用高...,"{ ""Course"": ""['Ch 1 开源大模型本地部署硬件指南.pdf']"", ""Mod..."
1,a4d419bf-68cd-47a7-9efb-bbdef5b8756e,['Ch 1 开源大模型本地部署硬件指南.pdf'],['data/Ch 1 开源大模型本地部署硬件指南.pdf'],一、大模型应用需求分析,"""硬件选择指南"" 或 ""配置优化要点""",大模型的本地部署主要应用于三个方面：训练（train）、高效微调（fine-tune）和推理...,本地部署大型模型主要涉及训练、高效微调和推理三个环节，其中算力消耗依次递减。由于从头训练大模...,"{ ""Course"": ""['Ch 1 开源大模型本地部署硬件指南.pdf']"", ""Mod..."
2,ad5870ff-83e3-4858-b9c5-613e7f2ad695,['Ch 1 开源大模型本地部署硬件指南.pdf'],['data/Ch 1 开源大模型本地部署硬件指南.pdf'],二、硬件配置的选择标准,"2.1 选择满足显存需求的 GPU, 2.2 主流显卡性能分析, 2.3 单卡4090 vs...",无论是个人使用、科研团队进行项目研究，还是企业寻求商业应用的落地，不同的应用场景和目标任务\...,本文介绍了选择GPU时需要考虑的因素，包括显存容量、算力、内存带宽等。NVIDIA的GPU在...,"{ ""Course"": ""['Ch 1 开源大模型本地部署硬件指南.pdf']"", ""Mod..."
3,96fe82ff-62b5-4a2d-bbe3-817854f96b22,['Ch 1 开源大模型本地部署硬件指南.pdf'],['data/Ch 1 开源大模型本地部署硬件指南.pdf'],三、组装计算机硬件选型策略,"3.1 GPU选型策略, 3.2 CPU选型策略, 3.3 散热选型策略, 3.5 硬盘选型...",计算机八大件：CPU、散热器、主板、内存、硬盘、显卡、电源、风扇、机箱，我们依次来看对于一套...,本文详细介绍了如何配置一台用于部署大模型的个人计算机，包括GPU、CPU、散热器、主板、内存...,"{ ""Course"": ""['Ch 1 开源大模型本地部署硬件指南.pdf']"", ""Mod..."
